# Homework 15 Webscrapping & API

## Task 1
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [7]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time


#####

KEYWORDS = ['code ', 'код']
# make KEYWORDS lowercase
KEYWORDS = list(map(lambda x: x.lower(), KEYWORDS))

link = 'https://habr.com/ru/all/'


#####

def get_full_habr_text_n_time(link):
    '''
    Takes a link to a Habr-article and returns full article content & creation time
    '''
    page = requests.get(link)
    html = BeautifulSoup(page.text, 'html.parser')
    text = html.find('article', class_='post_full').get_text().strip().lower()
    date_time = html.find('span', class_='post__time').get('data-time_published')
    return text, date_time


#####

result = pd.DataFrame()
page = requests.get(link)

html = BeautifulSoup(page.text, 'html.parser')
snippets = html.find_all('article', class_='post_preview')

for item in snippets:
    full_link = item.find('a', class_='post__title_link').get('href')
    # get full content (text data) and creation time
    full_text, date_time = get_full_habr_text_n_time(full_link)
    # if content contains any keywords from KEYWORDS
    if any(key in full_text for key in KEYWORDS):
        title = item.find('h2', class_='post__title').get_text().strip()
        date = pd.to_datetime(date_time).date()
        to_add = {'title': title, 'link': full_link, 'date': date, 'text': full_text}
        result = pd.concat([result, pd.DataFrame([to_add])])
    # just in case
    time.sleep(0.1)
    
result

,title,link,date,text
0,Зачем мне психотерапевт?,https://habr.com/ru/company/dododev/blog/532398/,2020-12-10,riskov\n\nсегодня в 20:07\n\n\nзачем мне психо...
0,Обобщаем паттерн посетитель (С++),https://habr.com/ru/post/532412/,2020-12-10,kutilkinfedor\n\nсегодня в 19:30\n\n\nобобщаем...
0,Почему сеньоры ненавидят собеседования с кодин...,https://habr.com/ru/company/skillfactory/blog/...,2020-12-10,honyaki\n\nсегодня в 19:25\n\n\nпочему сеньоры...
0,"Визуализация Пи, Тау и простых чисел",https://habr.com/ru/post/531050/,2020-12-10,yermack\n\nсегодня в 17:37\n\n\nвизуализация п...
0,Переброска данных между идентичными объектами ...,https://habr.com/ru/company/ivi/blog/532380/,2020-12-10,alexey_gru\n\nсегодня в 17:16\n\n\nпереброска ...
0,"История группы 414 — подростков из Милуоки, ко...",https://habr.com/ru/company/skillfactory/blog/...,2020-12-10,kd637\n\nсегодня в 17:14\n\n\nистория группы 4...
0,Space: публичный релиз командной среды от JetB...,https://habr.com/ru/company/JetBrains/blog/532...,2020-12-10,vandrianova\n\nсегодня в 16:56\n\n\nspace: пуб...
0,"Технологии и инструменты, на которые стоит обр...",https://habr.com/ru/company/ruvds/blog/530946/,2020-12-10,ru_vds\n\nсегодня в 16:15\n\n\nтехнологии и ин...
0,"Удалёнка кажется раем разработчика, но страдан...",https://habr.com/ru/post/532360/,2020-12-10,jonnygreenwoodtelecaster\n\nсегодня в 16:01\n\...
0,Обновление автомобильного софта по воздуху: пе...,https://habr.com/ru/company/itelma/blog/531684/,2020-12-10,itelma\n\nсегодня в 15:30\n\n\nобновление авто...


## Task 2

Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [12]:
import pandas as pd
import requests
import time


#EMAIL = ['orangutan@mail.ru', 'google@yandex.ru']
EMAIL = ['google@yandex.ru']


def get_hacks(email):
    '''
    Sends post request and gives back response from avast.com/hackcheck API for given e-mail address
    '''
    avast = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
    payload = {'emailAddresses': email}
    headers = {
        'Host': 'identityprotection.avast.com',
        'Connection': 'keep-alive',
        'Content-Length': '42',
        'Accept': 'application/json, text/plain, */*',
        'Vaar-Version': '0',
        'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.185 Mobile Safari/537.36',
        'Vaar-Header-App-Product': 'hackcheck-web-avast',
        'Content-Type': 'application/json;charset=UTF-8',
        'Origin': 'https://www.avast.com',
        'Sec-Fetch-Site': 'same-site',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
        'Referer': 'https://www.avast.com/hackcheck',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'ru,en;q=0.9'
    }
    return requests.post(avast, json=payload, headers=headers)


result = pd.DataFrame()
for email in EMAIL:
    resp = get_hacks(email)
    breaches = resp.json()['breaches']
    for key in breaches:
        date = pd.to_datetime((breaches[key]['publishDate'])).date()
        site = breaches[key]['site']
        desc = breaches[key]['description']
        to_add = {'email': email, 'date': date, 'source': site, 'description': desc}
        result = pd.concat([result, pd.DataFrame([to_add])])
    time.sleep(1)
    
result

null


In [6]:
print(resp)

<Response [200]>
